In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import re
import pickle
import os
from datetime import datetime, date, timedelta

import selenium

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
offers_db = pickle.load(open('db/offers_db', 'rb'))

In [3]:
offers_db

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
0,justjoin.it,junior,Software Developer Intern,AI Clearing,Warszawa,Not specified,6000.0,5000.0,7000.0,"[SQL, Python, Git]",https://justjoin.it/offers/ai-clearing-softwar...,2024-02-15 11:41:00,mazowieckie
1,justjoin.it,junior,AI/ML Engineer,MindPal,Warszawa,Praca zdalna,8500.0,7000.0,10000.0,"[AI, Machine Learning, C++]",https://justjoin.it/offers/mindpal-ai-ml-engin...,2024-02-15 11:41:00,mazowieckie
2,justjoin.it,junior,Junior Data Management Specialist,DXC Technology,Warszawa,Praca zdalna,NaN,NaN,NaN,"[data analytics, SQL, Python]",https://justjoin.it/offers/dxc-technology-juni...,2024-02-15 11:41:00,mazowieckie
3,justjoin.it,junior,Junior Python / DevOps Engineer,digatus,Warszawa,Praca zdalna,11000.0,9000.0,13000.0,"[Docker, DevOps, Python]",https://justjoin.it/offers/digatus-junior-pyth...,2024-02-15 11:41:00,mazowieckie
4,justjoin.it,junior,"Software Engineer, Early Career",Affirm,Warszawa,Not specified,16450.0,16450.0,16450.0,"[JavaScript, Kotlin, C++]",https://justjoin.it/offers/affirm-software-eng...,2024-02-15 11:41:00,mazowieckie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,pracuj.pl,senior,Business / Business-systems Analyst,Nexio Management Sp. z o.o.,3 lokalizacje,Praca hybrydowa,25200.0,21600.0,28800.0,"[SQL, Jira, Confluence]",https://pracodawcy.pracuj.pl/company/20012570,2024-03-18 10:30,Not specified
3134,pracuj.pl,mid,Specjalista ds. obsługi baz danych,Glen Mobile Solution,Kraków,Praca hybrydowa,6000.0,5000.0,7000.0,"[HTML, T-SQL, xml]",https://www.pracuj.pl/praca/specjalista-ds-obs...,2024-03-18 10:30,małopolskie
3135,pracuj.pl,mid,IT Business Analyst / Analityk Biznesowy IT,ArchiDoc S.A.,Chorzów,Praca hybrydowa,10000.0,8000.0,12000.0,"[SQL, UML, BPMN]",https://www.pracuj.pl/praca/it-business-analys...,2024-03-18 10:30,śląskie
3136,pracuj.pl,mid,Konsultant - Wdrożeniowiec systemów ERP,FMB Software,Gdynia,Praca hybrydowa,NaN,NaN,NaN,"[enova365, Symfonia, ERP]",https://www.pracuj.pl/praca/konsultant-wdrozen...,2024-03-18 10:30,pomorskie


In [4]:
offers_db['added_at'] = pd.to_datetime(offers_db['added_at'])

In [8]:
offers_db.sort_values('added_at', ascending=False).head(50)

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
2774,pracuj.pl,junior,Staż z Lwem - Zespół Analizy Danych HR,ING Bank Śląski S.A.,2 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,"[SQL, PowerBI, Excel]",https://pracodawcy.pracuj.pl/company/18797224,2024-03-10 11:52:00,Not specified
2770,pracuj.pl,mid,Specjalistka / Specjalista w Zespole ds. Rapor...,VeloBank S.A.,2 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,[],https://pracodawcy.pracuj.pl/company/20114475,2024-03-10 11:52:00,Not specified
2767,pracuj.pl,mid,Programista/ka Mainframe - obszar bankowości,Accenture,2 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,[COBOL],https://pracodawcy.pracuj.pl/company/20215732,2024-03-10 11:52:00,Not specified
2768,pracuj.pl,mid,Analityk Systemowy (k/m),Santander Bank Polska,3 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,[UML],https://pracodawcy.pracuj.pl/company/282,2024-03-10 11:52:00,Not specified
2769,pracuj.pl,c-level,IT Business Development Manager,ONWELO Sp. z o.o.,3 lokalizacje,Praca hybrydowa,14000.0,11000.0,17000.0,[],https://pracodawcy.pracuj.pl/company/20203737,2024-03-10 11:52:00,Not specified
2766,pracuj.pl,mid,Cloud Business Analyst,Ework Group,2 lokalizacje,Praca hybrydowa,23520.0,20160.0,26880.0,[],https://pracodawcy.pracuj.pl/company/20197100,2024-03-10 11:52:00,Not specified
2771,pracuj.pl,mid,Business and System Analyst,ALIOR BANK,3 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,"[UML, BPMN, Enterprise Architect, postgres, Or...",https://pracodawcy.pracuj.pl/company/20011599,2024-03-10 11:52:00,Not specified
2772,pracuj.pl,mid,IT Analyst (TM Exploration Team),emagine sp. z o.o.,4 lokalizacje,Praca hybrydowa,23680.0,23680.0,23680.0,"[Jira, Confluence]",https://pracodawcy.pracuj.pl/company/1074148163,2024-03-10 11:52:00,Not specified
2773,pracuj.pl,mid,IT Business Analyst LIMS,USP Zdrowie,2 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,[],https://pracodawcy.pracuj.pl/company/20086028,2024-03-10 11:52:00,Not specified
2516,justjoin.it,senior,Senior SQL Engineer,10 Ten Gaming LLC,Warszawa,Praca zdalna,28000.0,28000.0,28000.0,"[Azure, AWS, Redis]",https://justjoin.it/offers/10-ten-gaming-llc-s...,2024-03-10 11:27:00,mazowieckie


In [10]:
offers_db[offers_db['company']=='VeloBank S.A.']

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
894,pracuj.pl,senior,Ekspertka ds. Rozwoju CRM / Ekspert ds. Rozwoj...,VeloBank S.A.,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[SQL, SAS]",https://www.pracuj.pl/praca/ekspertka-ds-rozwo...,2024-02-15 11:41:00,mazowieckie
1514,pracuj.pl,mid,Analityk - biznesowo systemowy (SAS),VeloBank S.A.,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[SQL, SAS]",https://www.pracuj.pl/praca/analityk-biznesowo...,2024-02-22 10:56:00,mazowieckie
1618,pracuj.pl,mid,,VeloBank S.A.,2 lokalizacje,Praca zdalna,NaN,NaN,NaN,"[SQL, PLSQL, Git]",https://pracodawcy.pracuj.pl/company/20114475,2024-02-25 10:36:00,Not specified
1706,pracuj.pl,mid,Programistka Oracle / Programista Oracle / Ora...,VeloBank S.A.,2 lokalizacje,Praca zdalna,NaN,NaN,NaN,"[SQL, PLSQL, Git]",https://pracodawcy.pracuj.pl/company/20114475,2024-03-05 13:24:00,Not specified
1720,pracuj.pl,mid,Administratorka Baz Danych PostgreSQL / Admini...,VeloBank S.A.,Warszawa,Praca zdalna,NaN,NaN,NaN,"[SQL, PL/SQL, PostgreSQL, pgSQL]",https://www.pracuj.pl/praca/administratorka-ba...,2024-03-05 13:24:00,mazowieckie
1847,pracuj.pl,mid,Administratorka Linux / Administrator Linux,VeloBank S.A.,2 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,"[Ansible, Bash, Python, Linux]",https://pracodawcy.pracuj.pl/company/20114475,2024-03-06 14:16:00,Not specified
2016,pracuj.pl,mid,Specjalistka / Specjalista w Zespole Walidacji...,VeloBank S.A.,3 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,"[SQL, R]",https://pracodawcy.pracuj.pl/company/20114475,2024-03-06 14:16:00,Not specified
2770,pracuj.pl,mid,Specjalistka / Specjalista w Zespole ds. Rapor...,VeloBank S.A.,2 lokalizacje,Praca hybrydowa,NaN,NaN,NaN,[],https://pracodawcy.pracuj.pl/company/20114475,2024-03-10 11:52:00,Not specified


In [5]:
offers_db['added_at'] = pd.to_datetime(offers_db['added_at'])

# Filtrowanie wierszy z konkretną datą, na przykład '2024-02-15'
specific_date = pd.Timestamp('2024-03-18')
filtered_df = offers_db[offers_db['added_at'].dt.date >= specific_date.date()]

In [6]:
filtered_df

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
2806,justjoin.it,junior,Internship - Fullstack Python Developer Intern...,Samsung R&D Institute Poland,Warszawa,Not specified,NaN,NaN,NaN,"[GitHub, Python, Git]",https://justjoin.it/offers/samsung-r-d-institu...,2024-03-18 10:30:00,mazowieckie
2807,justjoin.it,junior,Internship - Security Infrastructure Intern,Samsung R&D Institute Poland,Warszawa,Not specified,NaN,NaN,NaN,[Python],https://justjoin.it/offers/samsung-r-d-institu...,2024-03-18 10:30:00,mazowieckie
2808,justjoin.it,junior,Fullstack Developer (Python / JS),DRUTEX S.A,Bytów,Not specified,7500.0,5000.0,10000.0,"[JS, Git, Python 3.x]",https://justjoin.it/offers/drutex-s-a-fullstac...,2024-03-18 10:30:00,pomorskie
2809,justjoin.it,mid,AI/LLM Engineer,QED Software,Warszawa,Not specified,16500.0,13000.0,20000.0,"[AI - DNN & LLM, OpenAI Rest API, Machine Lear...",https://justjoin.it/offers/qed-software-ai-llm...,2024-03-18 10:30:00,mazowieckie
2810,justjoin.it,mid,Data Scientist 💻📊,Adform,Warszawa,Not specified,20675.0,19700.0,21650.0,"[SQL, Analytical skills, Machine Learning]",https://justjoin.it/offers/adform-data-scienti...,2024-03-18 10:30:00,mazowieckie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3133,pracuj.pl,senior,Business / Business-systems Analyst,Nexio Management Sp. z o.o.,3 lokalizacje,Praca hybrydowa,25200.0,21600.0,28800.0,"[SQL, Jira, Confluence]",https://pracodawcy.pracuj.pl/company/20012570,2024-03-18 10:30:00,Not specified
3134,pracuj.pl,mid,Specjalista ds. obsługi baz danych,Glen Mobile Solution,Kraków,Praca hybrydowa,6000.0,5000.0,7000.0,"[HTML, T-SQL, xml]",https://www.pracuj.pl/praca/specjalista-ds-obs...,2024-03-18 10:30:00,małopolskie
3135,pracuj.pl,mid,IT Business Analyst / Analityk Biznesowy IT,ArchiDoc S.A.,Chorzów,Praca hybrydowa,10000.0,8000.0,12000.0,"[SQL, UML, BPMN]",https://www.pracuj.pl/praca/it-business-analys...,2024-03-18 10:30:00,śląskie
3136,pracuj.pl,mid,Konsultant - Wdrożeniowiec systemów ERP,FMB Software,Gdynia,Praca hybrydowa,NaN,NaN,NaN,"[enova365, Symfonia, ERP]",https://www.pracuj.pl/praca/konsultant-wdrozen...,2024-03-18 10:30:00,pomorskie


In [95]:
import sqlite3

In [96]:
DB_PATH = "db/offers.sqlite"

In [97]:
def create_db_if_not_exists():
    create_offers_table = """
    CREATE TABLE IF NOT EXISTS offers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    site TEXT NOT NULL,
    experience TEXT NOT NULL, 
    name TEXT NOT NULL,
    company TEXT NOT NULL, 
    location TEXT,
    work_mode TEXT,
    salary_avg FLOAT,
    salary_low FLOAT,
    salary_high FLOAT,
    technologies TEXT,
    link TEXT,
    added_at TEXT,
    voivodeship TEXT
    );
    """
    
    if os.path.exists(DB_PATH):
        print("Succesfully connected to Database")
    else:
        with sqlite3.connect(DB_PATH) as connection:
            cursor = connection.cursor()
            cursor.execute(create_offers_table)
            connection.commit()
        print("New database created")

In [98]:
create_db_if_not_exists()

In [99]:
def df_to_db(offers: pd.DataFrame):
    offers['technologies'] = offers['technologies'].apply(lambda row: json.dumps(row))
    offers['added_at'] = offers['added_at'].astype(str)
    db_data = list(zip(*[offers[column].tolist() for column in offers.columns]))
    
    add_offer_to_db = """
    INSERT INTO 
        offers (site, experience, name, company, location, work_mode, salary_avg, 
                salary_low, salary_high, technologies, link, added_at, voivodeship)
    VALUES
        (?,?,?,?,?,?,?,?,?,?,?,?,?)
    """
    with sqlite3.connect(DB_PATH) as connection:
        cursor = connection.cursor()
        cursor.executemany(add_offer_to_db, db_data)
        connection.commit()

In [100]:
sample = offers_db.tail(50)

In [106]:
sample

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
3088,pracuj.pl,junior,Trainee IT - Graduate Program,Parker Hannifin ESSC Sp. z o.o.,Wrocław,Praca stacjonarna,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/trainee-it-graduat...,2024-03-18 10:30,dolnośląskie
3089,pracuj.pl,mid,Analityk systemowy ds. infrastruktury w InsERT GT,Insert S.A.,Wrocław,Praca hybrydowa,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/analityk-systemowy...,2024-03-18 10:30,dolnośląskie
3090,pracuj.pl,mid,Młodszy Analityk IT,North.pl spółka z ograniczoną odpowiedzialnośc...,Koszalin,Praca hybrydowa,NaN,NaN,NaN,"[""Jira"", ""Confluence""]",https://www.pracuj.pl/praca/mlodszy-analityk-i...,2024-03-18 10:30,zachodniopomorskie
3091,pracuj.pl,mid,Machine Learning Engineer,CONCIERGEONLINE. COM SPÓŁKA Z OGRANICZONĄ ODPO...,Kraków,Praca zdalna,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/machine-learning-e...,2024-03-18 10:30,małopolskie
3092,pracuj.pl,c-level,Head of Data Platform & Product Analytics,XTB S.A.,Warszawa,Praca hybrydowa,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/head-of-data-platf...,2024-03-18 10:30,mazowieckie
3093,pracuj.pl,mid,Analityk systemowy,TUZ Towarzystwo Ubezpieczeń Wzajemnych,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[""Jira"", ""Confluence""]",https://www.pracuj.pl/praca/analityk-systemowy...,2024-03-18 10:30,mazowieckie
3094,pracuj.pl,mid,Conversational AI Backend Developer,Capgemini Polska,Kraków,Praca hybrydowa,NaN,NaN,NaN,"[""API key"", ""OAuth"", ""Dialogflow CX"", ""Google ...",https://www.pracuj.pl/praca/conversational-ai-...,2024-03-18 10:30,małopolskie
3095,pracuj.pl,mid,Automation |RPA| Developer - Blue Prism,Capgemini Polska,Kraków,Praca hybrydowa,NaN,NaN,NaN,"[""RPA""]",https://www.pracuj.pl/praca/automation-rpa-dev...,2024-03-18 10:30,małopolskie
3096,pracuj.pl,mid,Analityk Rozwiązań GIS/ESRI,GISonLine Sp. z o.o.,Kraków,Praca hybrydowa,NaN,NaN,NaN,"[""ArcGIS Pro"", ""Portal"", ""ArcGIS Server""]",https://www.pracuj.pl/praca/analityk-rozwiazan...,2024-03-18 10:30,małopolskie
3097,pracuj.pl,senior,Starszy Analityk Danych Systemów Logistycznych,ILS Sp. z o.o.,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[""MS Reporting Services"", ""SQL"", ""Oracle""]",https://www.pracuj.pl/praca/starszy-analityk-d...,2024-03-18 10:30,mazowieckie


In [101]:
df_to_db(sample)

C:\Users\GMR\AppData\Local\Temp\ipykernel_9748\2833747822.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offers['technologies'] = offers['technologies'].apply(lambda row: json.dumps(row))
C:\Users\GMR\AppData\Local\Temp\ipykernel_9748\2833747822.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  offers['added_at'] = offers['added_at'].astype(str)


In [104]:
with sqlite3.connect(DB_PATH) as connection:
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM offers")
    offers = cursor.fetchall()
    
    for row in offers:
        print(row)

(1, 'pracuj.pl', 'junior', 'Trainee IT - Graduate Program', 'Parker Hannifin ESSC Sp. z o.o.', 'Wrocław', 'Praca stacjonarna', None, None, None, '[]', 'https://www.pracuj.pl/praca/trainee-it-graduate-program-wroclaw-stanislawa-dubois-41,oferta,1003218815?s=c222eb4a&searchId=MTcxMDc1NDIwODQxMy41Mzc0', '2024-03-18 10:30', 'dolnośląskie')
(2, 'pracuj.pl', 'mid', 'Analityk systemowy ds. infrastruktury w InsERT GT', 'Insert S.A.', 'Wrocław', 'Praca hybrydowa', None, None, None, '[]', 'https://www.pracuj.pl/praca/analityk-systemowy-ds-infrastruktury-w-insert-gt-wroclaw-jerzmanowska-2,oferta,1003218814?s=c222eb4a&searchId=MTcxMDc1NDIwODQxMy41Mzc0', '2024-03-18 10:30', 'dolnośląskie')
(3, 'pracuj.pl', 'mid', 'Młodszy Analityk IT', 'North.pl spółka z ograniczoną odpowiedzialnością sp.k.', 'Koszalin', 'Praca hybrydowa', None, None, None, '["Jira", "Confluence"]', 'https://www.pracuj.pl/praca/mlodszy-analityk-it-koszalin-wawozowa-7b,oferta,1003218797?s=c222eb4a&searchId=MTcxMDc1NDIwODQxMy41Mzc0',

In [129]:
def db_to_df():
    select_query = """
        SELECT 
            site, experience, name, company, location, work_mode, salary_avg, 
            salary_low, salary_high, technologies, link, added_at, voivodeship
        FROM offers
        """

    with sqlite3.connect(DB_PATH) as connection:

        db_df = pd.read_sql_query(query, connection)
        db_df['technologies'] = db_df['technologies'].apply(lambda row: json.loads(row))
        db_df['added_at'] = pd.to_datetime(db_df['added_at'])
    
    return db_df

In [130]:
offers_db = db_to_df()

In [131]:
offers_db

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship
0,pracuj.pl,junior,Trainee IT - Graduate Program,Parker Hannifin ESSC Sp. z o.o.,Wrocław,Praca stacjonarna,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/trainee-it-graduat...,2024-03-18 10:30:00,dolnośląskie
1,pracuj.pl,mid,Analityk systemowy ds. infrastruktury w InsERT GT,Insert S.A.,Wrocław,Praca hybrydowa,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/analityk-systemowy...,2024-03-18 10:30:00,dolnośląskie
2,pracuj.pl,mid,Młodszy Analityk IT,North.pl spółka z ograniczoną odpowiedzialnośc...,Koszalin,Praca hybrydowa,NaN,NaN,NaN,"[Jira, Confluence]",https://www.pracuj.pl/praca/mlodszy-analityk-i...,2024-03-18 10:30:00,zachodniopomorskie
3,pracuj.pl,mid,Machine Learning Engineer,CONCIERGEONLINE. COM SPÓŁKA Z OGRANICZONĄ ODPO...,Kraków,Praca zdalna,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/machine-learning-e...,2024-03-18 10:30:00,małopolskie
4,pracuj.pl,c-level,Head of Data Platform & Product Analytics,XTB S.A.,Warszawa,Praca hybrydowa,NaN,NaN,NaN,[],https://www.pracuj.pl/praca/head-of-data-platf...,2024-03-18 10:30:00,mazowieckie
5,pracuj.pl,mid,Analityk systemowy,TUZ Towarzystwo Ubezpieczeń Wzajemnych,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[Jira, Confluence]",https://www.pracuj.pl/praca/analityk-systemowy...,2024-03-18 10:30:00,mazowieckie
6,pracuj.pl,mid,Conversational AI Backend Developer,Capgemini Polska,Kraków,Praca hybrydowa,NaN,NaN,NaN,"[API key, OAuth, Dialogflow CX, Google Cloud s...",https://www.pracuj.pl/praca/conversational-ai-...,2024-03-18 10:30:00,małopolskie
7,pracuj.pl,mid,Automation |RPA| Developer - Blue Prism,Capgemini Polska,Kraków,Praca hybrydowa,NaN,NaN,NaN,[RPA],https://www.pracuj.pl/praca/automation-rpa-dev...,2024-03-18 10:30:00,małopolskie
8,pracuj.pl,mid,Analityk Rozwiązań GIS/ESRI,GISonLine Sp. z o.o.,Kraków,Praca hybrydowa,NaN,NaN,NaN,"[ArcGIS Pro, Portal, ArcGIS Server]",https://www.pracuj.pl/praca/analityk-rozwiazan...,2024-03-18 10:30:00,małopolskie
9,pracuj.pl,senior,Starszy Analityk Danych Systemów Logistycznych,ILS Sp. z o.o.,Warszawa,Praca hybrydowa,NaN,NaN,NaN,"[MS Reporting Services, SQL, Oracle]",https://www.pracuj.pl/praca/starszy-analityk-d...,2024-03-18 10:30:00,mazowieckie


In [137]:
connection.close()

In [128]:
type(db_df['technologies'].iloc[49])

list

In [122]:
type(offers_db['added_at'].iloc[49])

pandas._libs.tslibs.timestamps.Timestamp

In [80]:
offers_db[offers_db['name'].isna()]

,site,experience,name,company,location,work_mode,salary_avg,salary_low,salary_high,technologies,link,added_at,voivodeship


In [19]:
with sqlite3.connect(DB_PATH) as connection:
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM offers")
    offers = cursor.fetchall()
    print(cursor.description)

(('id', None, None, None, None, None, None), ('site', None, None, None, None, None, None), ('experience', None, None, None, None, None, None), ('name', None, None, None, None, None, None), ('company', None, None, None, None, None, None), ('location', None, None, None, None, None, None), ('work_mode', None, None, None, None, None, None), ('salary_avg', None, None, None, None, None, None), ('salary_low', None, None, None, None, None, None), ('salary_high', None, None, None, None, None, None), ('technologies', None, None, None, None, None, None), ('link', None, None, None, None, None, None), ('added_at', None, None, None, None, None, None), ('voivodeship', None, None, None, None, None, None))


In [28]:
connection.close()

In [52]:
sample = json.dumps(offers_db['technologies'].iloc[0])

In [94]:
json.loads(sample)

TypeError: the JSON object must be str, bytes or bytearray, not DataFrame

In [58]:
sample

'["SQL", "Python", "Git"]'